In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

MP_API_KEY = os.getenv("MP_API_KEY")

In [2]:
from pymatgen.core import Structure
from pymatgen.io.vasp.sets import MPStaticSet

In [3]:
from mp_api.client import MPRester

with MPRester() as mpr:

    docs = mpr.materials.summary.search(
        chemsys="Ba-Ti-O",
        # material_ids=["mp-5777"],
        is_metal=False,
        band_gap=(2, None),
        fields=["structure", "material_id", "symmetry"],
    )
    

Retrieving SummaryDoc documents:   0%|          | 0/20 [00:00<?, ?it/s]

In [4]:
import yaml
from pathlib import Path
from atomate2.settings import Atomate2Settings
from atomate2.vasp.jobs.core import RelaxMaker, StaticMaker
from atomate2.vasp.jobs.base import BaseVaspMaker
from jobflow import run_locally
from pymatgen.io.vasp.sets import MatPESStaticSet, MPStaticSet, MP24StaticSet
from atomate2.vasp.sets.base import VaspInputGenerator, _BASE_VASP_SET
from atomate2.vasp.sets.core import StaticSetGenerator
from pymatgen.io.vasp import Kpoints

structure = docs[4].structure # * (3, 3, 3)

atomate2_config_file = Path(
    f"~/atomate2/config/atomate2-test.yaml"
).expanduser()
atomate2_config_file.parent.mkdir(parents=True, exist_ok=True)

vasp_prefix = Path("/global/home/users/cyrusyc/.local/bin")
num_processes = 2

with open(atomate2_config_file, "w") as f:
    yaml.dump(
        {
            "VASP_CMD": f"mpirun -np {num_processes} {vasp_prefix / 'vasp_std'}",
            "VASP_GAMMA_CMD": f"mpirun -np {num_processes} {vasp_prefix / 'vasp_gam'}",
            # "VASP_CMD": f"srun --export=ALL,LD_LIBRARY_PATH={os.environ['LD_LIBRARY_PATH']} -n 1 -c 4 -G 1 {vasp_prefix / 'vasp_std'}",
            # "VASP_GAMMA_CMD": f"srun --export=ALL,LD_LIBRARY_PATH={os.environ['LD_LIBRARY_PATH']} -n 1 -c 4 -G 1  {vasp_prefix / 'vasp_gam'}",
        },
        f,
    )

# pre_run = """
# module load nvhpc/23.11;
# export LD_LIBRARY_PATH=/global/software/rocky-8.x86_64/gcc/linux-rocky8-x86_64/gcc-8.5.0/nvhpc-23.11-gh5cygvdqksy6mxuy2xgoibowwxi3w7t/Linux_x86_64/23.11/compilers/extras/qd/lib:/global/software/rocky-8.x86_64/gcc/linux-rocky8-x86_64/gcc-8.5.0/nvhpc-23.11-gh5cygvdqksy6mxuy2xgoibowwxi3w7t/Linux_x86_64/23.11/cuda/lib64:$LD_LIBRARY_PATH
# """

# pre_run = """export LD_LIBRARY_PATH=/global/software/rocky-8.x86_64/gcc/linux-rocky8-x86_64/gcc-8.5.0/nvhpc-23.11-gh5cygvdqksy6mxuy2xgoibowwxi3w7t/Linux_x86_64/23.11/compilers/extras/qd/lib:/global/software/rocky-8.x86_64/gcc/linux-rocky8-x86_64/gcc-8.5.0/nvhpc-23.11-gh5cygvdqksy6mxuy2xgoibowwxi3w7t/Linux_x86_64/23.11/cuda/lib64:$LD_LIBRARY_PATH"""

os.environ["LD_LIBRARY_PATH"] = "/global/software/rocky-8.x86_64/gcc/linux-rocky8-x86_64/gcc-8.5.0/nvhpc-23.11-gh5cygvdqksy6mxuy2xgoibowwxi3w7t/Linux_x86_64/23.11/compilers/extras/qd/lib:/global/software/rocky-8.x86_64/gcc/linux-rocky8-x86_64/gcc-8.5.0/nvhpc-23.11-gh5cygvdqksy6mxuy2xgoibowwxi3w7t/Linux_x86_64/23.11/cuda/lib64:$LD_LIBRARY_PATH"
# os.environ["OMP_NUM_THREADS"] = "1"
# os.environ["OMP_PLACES"] = "threads"
# os.environ["OMP_PROC_BIND"] = "spread"

settings = Atomate2Settings(
    CONFIG_FILE=str(atomate2_config_file),
    # VASP_CMD=f"mpirun -np {num_processes} {vasp_prefix / 'vasp_std'}",
    # VASP_GAMMA_CMD=f"mpirun -np {num_processes} {vasp_prefix / 'vasp_gam'}",
)

user_incar_settings = {
    "ALGO": "Fast",
    "EDIFF": 1e-5,
    "ISMEAR": -5,
    # "SIGMA": 0.05,
    "LCALCPOL": True,
    "LCALCEPS": True,
    "EFIELD_PEAD": [0.0036, 0.0036, 0.0036], 
    "LREAL": "Auto",
    "KPAR": 4,
    "NCORE": 1,
    "NSIM": 8
}

input_set_generator = StaticSetGenerator(
    user_incar_settings=user_incar_settings,
    # user_kpoints_settings=Kpoints(), #{"reciprocal_density": 10},
    user_potcar_functional="PBE_64",
    # auto_ismear=False
)

# input_set_generator.config_dict["INCAR"]["ISMEAR"] = -5
# input_set_generator.auto_ismear = False

structure.site_properties.pop("magmom")

job = BaseVaspMaker(
    input_set_generator=input_set_generator,
    run_vasp_kwargs=dict(
        vasp_cmd=settings.VASP_CMD,
        vasp_gamma_cmd=settings.VASP_GAMMA_CMD,
    ),
).make(structure)


# run the job
run_locally(job, create_folders=True)


2025-02-19 17:30:08,778 INFO Started executing jobs locally
2025-02-19 17:30:08,792 INFO Starting job - base vasp job (8c5044d7-8006-4aec-b2fb-c59454df3aa9)


<string>:33: BadInputSetWarning: Overriding the POTCAR functional is generally not recommended  as it significantly affects the results of calculations and compatibility with other calculations done with the same input set. Note that some POTCAR symbols specified in the configuration file may not be available in the selected functional.
/global/home/users/cyrusyc/.conda/envs/chace/lib/python3.12/site-packages/custodian/vasp/handlers.py:185: EncodingWarning: We strongly encourage explicit `encoding`, and we would use UTF-8 by default as per PEP 686
  with zopen(os.path.join(directory, self.output_filename), mode="rt") as file:


In [5]:
import yaml
from ase.calculators.vasp import Vasp

structure = docs[4].structure

os.environ["VASP_PP_PATH"] = "/global/home/users/cyrusyc/potcars"

incar = input_set_generator.get_input_set(structure=structure)['INCAR']

incar["ISMEAR"] = -5
incar["SIGMA"] = 0.1

calc = Vasp(
    command=f'mpirun -np {num_processes} {vasp_prefix / 'vasp_gam'}',
    **incar,
)

calc.get_potential_energy(structure.to_ase_atoms(msonable=False))


RuntimeError: Looking for PP for potpaw/Ba_sv/POTCAR
                        The pseudopotentials are expected to be in:
                        LDA:  $VASP_PP_PATH/potpaw/
                        PBE:  $VASP_PP_PATH/potpaw_PBE/
                        PW91: $VASP_PP_PATH/potpaw_GGA/

                        No pseudopotential for Ba!